In [61]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[ 
        StreamingStdOutCallbackHandler(),
    ]
)

In [63]:
chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
        ("human", "I want to cook {cuisine} food.")
    ]
)

chef_chain = chef_prompt | chat

In [64]:
# 생성된 레시피를 채식으로 변경하는 prompt (대체품이 없다면 레시피를 모른다 말하세요 -> 할루시네이션 방지를 위한 프롬포트)
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
        ("human", "{recipe}")
    ]
)
veg_chain = veg_chef_prompt | chat

In [66]:
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "제육볶음"
})

제육볶음 (Jeyuk Bokkeum) is a popular Korean dish made with thinly sliced pork belly stir-fried with vegetables in a spicy and savory sauce. Here's a simple recipe for you to try at home:

Ingredients:
- 500g pork belly, thinly sliced
- 1 onion, sliced
- 1 carrot, julienned
- 3-4 green onions, chopped
- 4 cloves of garlic, minced
- 2 tbsp soy sauce
- 1 tbsp gochujang (Korean red chili paste)
- 1 tbsp gochugaru (Korean red chili flakes)
- 1 tbsp sugar
- 1 tbsp sesame oil
- 1 tbsp vegetable oil
- Sesame seeds for garnish
- Cooked rice, to serve

Instructions:
1. In a bowl, mix together soy sauce, gochujang, gochugaru, sugar, and sesame oil to make the marinade.
2. Add the sliced pork belly to the marinade and mix well. Let it marinate for at least 30 minutes.
3. Heat vegetable oil in a large pan or wok over medium-high heat.
4. Add the minced garlic and stir-fry for about 30 seconds until fragrant.
5. Add the marinated pork belly and stir-fry for 5-7 minutes until the pork is cooked through.

AIMessageChunk(content="To make a vegetarian version of 제육볶음 (Jeyuk Bokkeum), you can substitute the pork belly with a plant-based alternative such as seitan or tempeh. Here's how you can adapt the recipe:\n\nIngredients:\n- 500g plant-based protein (seitan or tempeh), thinly sliced\n- 1 onion, sliced\n- 1 carrot, julienned\n- 3-4 green onions, chopped\n- 4 cloves of garlic, minced\n- 2 tbsp soy sauce\n- 1 tbsp gochujang (Korean red chili paste)\n- 1 tbsp gochugaru (Korean red chili flakes)\n- 1 tbsp sugar\n- 1 tbsp sesame oil\n- 1 tbsp vegetable oil\n- Sesame seeds for garnish\n- Cooked rice, to serve\n\nInstructions:\n1. In a bowl, mix together soy sauce, gochujang, gochugaru, sugar, and sesame oil to make the marinade.\n2. Add the sliced plant-based protein to the marinade and mix well. Let it marinate for at least 30 minutes.\n3. Heat vegetable oil in a large pan or wok over medium-high heat.\n4. Add the minced garlic and stir-fry for about 30 seconds until fragrant.\n5. Add the ma